In [27]:
import pandas as pd
import re
import datetime as dt


In [28]:
# read 2nd sheet of an excel file
ddid = pd.read_excel(r"/Users/brentcleland/NSS/Projects/ddid-rasberry-pi/data/TEIS Exit Data for NSS.xlsx", sheet_name =3)

In [3]:
ddid.head()

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and ..."
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic a...


In [29]:
ddid[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date', 'Exit Reason', 'Exit Note']]

,Child ID,DOB,Initial IFSP Date,Exit Date,Exit Reason,Exit Note
0,471738,2019-02-28,2019-10-31,2020-09-21,618 - Parent withdraw,* No Exit ECO available due to COVID-19
1,449030,2018-07-22,2018-12-18,2021-03-06,618 - Moved out of state,"***family moving out of state March 2021, and ..."
2,463311,2017-07-09,2019-07-02,2020-07-08,618 - Part B eligibility not determined,**Completed the transition planning conference...
3,463313,2017-07-09,2019-07-02,2020-07-08,618 - Part B eligibility not determined,**Completed the transition planning conference...
4,469194,2017-08-31,2019-10-11,2020-08-30,618 - Part B eligibility not determined,**Due to the impact of the COVID-19 pandemic a...
...,...,...,...,...,...,...
5577,491609,2018-12-22,2020-11-09,2021-05-17,618 - Attempts to contact unsuccessful,NaN
5578,491743,2018-06-05,2020-11-09,2021-06-04,618 - Part B eligibility not determined,NaN
5579,491879,2019-04-24,2020-11-04,2021-05-06,618 - Parent withdraw,NaN
5580,492267,2018-06-01,2020-11-24,2021-05-31,618 - Part B eligibility not determined,NaN


In [30]:
ddid_filtered = ddid[ddid['Exit Note'].notnull()]
ddid_filtered.head()

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and ..."
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic a...


In [31]:
ddid_filtered['Exit Note']

0                 * No Exit ECO available due to COVID-19
1       ***family moving out of state March 2021, and ...
2       **Completed the transition planning conference...
3       **Completed the transition planning conference...
4       **Due to the impact of the COVID-19 pandemic a...
                              ...                        
2246    Transition meeting late due to COVID-19 Pandem...
2247    Transition meeting late due to COVID-19 Pandem...
2248    Transition meeting was scheduled for 5/4/20 bu...
2249                          Child is turning 3 yrs old.
2250             Unable to contact Closed file: 4/16/2021
Name: Exit Note, Length: 2251, dtype: object

In [ ]:
#ddid_filtered.loc[ddid_filtered['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]


In [7]:
ddid_filtered.to_csv('new_filter.csv')

In [32]:
search = ddid_filtered.loc[ddid_filtered['Exit Note'].str.contains('(Reopened:)\s\d{2}/\d{2}/\d{4}\s(Exit Date:)\s\d{2}/\d{2}/\d{4}')]
search.head()

/Users/brentcleland/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...
12,SE,436339,2018-04-05,2019-01-11,2019-01-31,2019-02-13,2021-01-29,618 - Part B eligibility not determined,2021-04-01,778,04/01/2021- File closed business day prior to ...
16,ET,474102,2018-05-11,2020-02-27,2020-03-12,2020-04-09,2021-03-29,618 - Part B eligibility not determined,2021-05-10,396,05/10/2021 TPC held then parents withdrew from...
21,GN,476958,2018-10-17,2020-08-04,2020-08-18,2020-08-24,2021-02-12,618 - Parent withdraw,2021-05-25,274,05/25/21- parent withdraw Reopened: 08/04/2020...


In [33]:
search['Exit Note'].str.findall(r'Reopened:\d{2}/\d{2}/\d{4}')
search.head(2)

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...


In [34]:
dates = search['Exit Note'].str.findall(r'(Reopened: \d{2}/\d{2}/\d{4}) (Exit Date: \d{2}/\d{2}/\d{4})')
dates

5       [(Reopened: 04/06/2020, Exit Date: 11/26/2019)]
8       [(Reopened: 02/05/2021, Exit Date: 04/07/2020)]
12      [(Reopened: 01/11/2019, Exit Date: 06/17/2018)]
16      [(Reopened: 02/28/2020, Exit Date: 12/05/2019)]
21      [(Reopened: 08/04/2020, Exit Date: 03/09/2020)]
                             ...                       
2223    [(Reopened: 07/28/2020, Exit Date: 04/26/2020)]
2227    [(Reopened: 02/05/2019, Exit Date: 10/23/2018)]
2232    [(Reopened: 03/05/2020, Exit Date: 09/17/2019)]
2233    [(Reopened: 10/25/2018, Exit Date: 05/24/2018)]
2243    [(Reopened: 07/19/2019, Exit Date: 06/25/2019)]
Name: Exit Note, Length: 1691, dtype: object

In [17]:
reopen_adjusted = search['Exit Note'].str.findall(r'(Reopened: \d{2}/\d{2}/\d{4})')
reopen_adjusted

5       [Reopened: 04/06/2020]
8       [Reopened: 02/05/2021]
12      [Reopened: 01/11/2019]
16      [Reopened: 02/28/2020]
21      [Reopened: 08/04/2020]
                 ...          
2223    [Reopened: 07/28/2020]
2227    [Reopened: 02/05/2019]
2232    [Reopened: 03/05/2020]
2233    [Reopened: 10/25/2018]
2243    [Reopened: 07/19/2019]
Name: Exit Note, Length: 1691, dtype: object

In [36]:
exit_adjusted  =search['Exit Note'].str.findall(r'(Exit Date: \d{2}/\d{2}/\d{4})')
exit_adjusted

5       [Exit Date: 11/26/2019]
8       [Exit Date: 04/07/2020]
12      [Exit Date: 06/17/2018]
16      [Exit Date: 12/05/2019]
21      [Exit Date: 03/09/2020]
                 ...           
2223    [Exit Date: 04/26/2020]
2227    [Exit Date: 10/23/2018]
2232    [Exit Date: 09/17/2019]
2233    [Exit Date: 05/24/2018]
2243    [Exit Date: 06/25/2019]
Name: Exit Note, Length: 1691, dtype: object

In [37]:
exit_adjusted1  =search['Exit Note'].str.findall(r'\d{2}/\d{2}/\d{4}')
exit_adjusted1

5                   [04/06/2020, 11/26/2019]
8                   [02/05/2021, 04/07/2020]
12      [04/01/2021, 01/11/2019, 06/17/2018]
16      [05/10/2021, 02/28/2020, 12/05/2019]
21                  [08/04/2020, 03/09/2020]
                        ...                 
2223                [07/28/2020, 04/26/2020]
2227                [02/05/2019, 10/23/2018]
2232    [10/19/2020, 03/05/2020, 09/17/2019]
2233                [10/25/2018, 05/24/2018]
2243                [07/19/2019, 06/25/2019]
Name: Exit Note, Length: 1691, dtype: object

In [19]:
search['reopen_date'] = reopen_adjusted
search.head()

<ipython-input-19-af3721c21b5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search['reopen_date'] = reopen_adjusted


,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,reopen_date
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...,[Reopened: 04/06/2020]
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...,[Reopened: 02/05/2021]
12,SE,436339,2018-04-05,2019-01-11,2019-01-31,2019-02-13,2021-01-29,618 - Part B eligibility not determined,2021-04-01,778,04/01/2021- File closed business day prior to ...,[Reopened: 01/11/2019]
16,ET,474102,2018-05-11,2020-02-27,2020-03-12,2020-04-09,2021-03-29,618 - Part B eligibility not determined,2021-05-10,396,05/10/2021 TPC held then parents withdrew from...,[Reopened: 02/28/2020]
21,GN,476958,2018-10-17,2020-08-04,2020-08-18,2020-08-24,2021-02-12,618 - Parent withdraw,2021-05-25,274,05/25/21- parent withdraw Reopened: 08/04/2020...,[Reopened: 08/04/2020]


In [35]:
search['exit_date_adjusted'] = exit_adjusted
search.head()

<ipython-input-35-060a63fe7047>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search['exit_date_adjusted'] = exit_adjusted


,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,exit_date_adjusted
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...,[Exit Date: 11/26/2019]
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...,[Exit Date: 04/07/2020]
12,SE,436339,2018-04-05,2019-01-11,2019-01-31,2019-02-13,2021-01-29,618 - Part B eligibility not determined,2021-04-01,778,04/01/2021- File closed business day prior to ...,[Exit Date: 06/17/2018]
16,ET,474102,2018-05-11,2020-02-27,2020-03-12,2020-04-09,2021-03-29,618 - Part B eligibility not determined,2021-05-10,396,05/10/2021 TPC held then parents withdrew from...,[Exit Date: 12/05/2019]
21,GN,476958,2018-10-17,2020-08-04,2020-08-18,2020-08-24,2021-02-12,618 - Parent withdraw,2021-05-25,274,05/25/21- parent withdraw Reopened: 08/04/2020...,[Exit Date: 03/09/2020]


In [25]:
search['exit_date_adjusted'].str.findall(r'\d{2}/\d{2}/\d{4}')


5      NaN
8      NaN
12     NaN
16     NaN
21     NaN
        ..
2223   NaN
2227   NaN
2232   NaN
2233   NaN
2243   NaN
Name: exit_date_adjusted, Length: 1691, dtype: float64